In [1]:
import os
%env SPECS_DIR=/workspace/tao-experiments/lpd/specs
%env EXPERIMENT_DIR=/workspace/tao-experiments/lpd/results
%env TFRECORDS = /workspace/tao-experiments/lpd/tfrecords
%env KEY=nvidia_tlt
%env GPU_INDEX=0
os.environ['LOCAL_PROJECT_DIR']='/home/pdelafuente/tao-experiments/lpd'
os.environ['LOCAL_DATA_DIR']='/home/pdelafuente/tao-experiments/lpd/lpd/data'
os.environ['DATA_DIR']='/home/pdelafuente/tao-experiments/lpd/lpd/data'
os.environ['MODELS_DIR']='/home/pdelafuente/tao-experiments/lpd/models'
os.environ['LOCAL_SPECS_DIR']='/home/pdelafuente/tao-experiments/lpd/specs'
os.environ['LOCAL_EXPERIMENT_DIR']='/home/pdelafuente/tao-experiments/lpd/results'


!mkdir $LOCAL_PROJECT_DIR
!mkdir $LOCAL_DATA_DIR
!mkdir $MODELS_DIR
!mkdir $LOCAL_EXPERIMENT_DIR
!mkdir $LOCAL_SPECS_DIR
!mkdir $TFRECORDS

env: SPECS_DIR=/workspace/tao-experiments/lpd/specs
env: EXPERIMENT_DIR=/workspace/tao-experiments/lpd/results
env: TFRECORDS=/workspace/tao-experiments/lpd/tfrecords
env: KEY=nvidia_tlt
env: GPU_INDEX=0
mkdir: cannot create directory ‘/home/pdelafuente/tao-experiments/lpd’: File exists
mkdir: cannot create directory ‘/home/pdelafuente/tao-experiments/lpd/lpd/data’: File exists
mkdir: cannot create directory ‘/home/pdelafuente/tao-experiments/lpd/models’: File exists
mkdir: cannot create directory ‘/home/pdelafuente/tao-experiments/lpd/results’: File exists
mkdir: cannot create directory ‘/home/pdelafuente/tao-experiments/lpd/specs’: File exists
mkdir: cannot create directory ‘/workspace/tao-experiments/lpd/tfrecords’: No such file or directory


In [2]:
# Mapping up the local directories to the TAO docker.
import json
mounts_file = os.path.expanduser("~/.tao_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
     "Mounts": [
            # Mapping the data directory
            {
                "source": os.environ["LOCAL_PROJECT_DIR"],
                "destination": "/workspace/tao-experiments/lpd"
            },
            # Mapping the specs directory.
            {
                "source": os.environ["LOCAL_SPECS_DIR"],
                "destination": os.environ["SPECS_DIR"]
            },
         {
                "source": os.environ["LOCAL_EXPERIMENT_DIR"],
                "destination": os.environ["EXPERIMENT_DIR"]
            },
        ]
}

# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [3]:
!cat ~/.tao_mounts.json

{
    "Mounts": [
        {
            "source": "/home/pdelafuente/tao-experiments/lpd",
            "destination": "/workspace/tao-experiments/lpd"
        },
        {
            "source": "/home/pdelafuente/tao-experiments/lpd/specs",
            "destination": "/workspace/tao-experiments/lpd/specs"
        },
        {
            "source": "/home/pdelafuente/tao-experiments/lpd/results",
            "destination": "/workspace/tao-experiments/lpd/results"
        }
    ]
}

In [12]:
!git clone https://github.com/openalpr/benchmarks benchmarks

fatal: destination path 'benchmarks' already exists and is not an empty directory.


In [13]:
"""Script to prepare resized images/labels for LPD."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import cv2
from PIL import Image

input_dir = "benchmarks/endtoend/us" 
output_dir = "lpd"  
target_width =  640 
target_height = 480



def prepare_data(input_dir, img_list, output_dir, output_size):
    """Crop the license plates from the orginal images."""
    w, h = output_size
    target_img_path = os.path.join(output_dir, "image")
    target_label_path = os.path.join(output_dir, "label")

    if not os.path.exists(target_img_path):
        os.makedirs(target_img_path)

    if not os.path.exists(target_label_path):
        os.makedirs(target_label_path)

    for img_name in img_list:
        img_path = os.path.join(input_dir, img_name)
        label_path = os.path.join(input_dir,
                                  img_name.split(".")[0] + ".txt")
        print("Resizing {} and its label".format(img_path))

        # resize labels
        img = cv2.imread(img_path)
        (height, width, _) = img.shape
        ratio_w = float(float(w)/float(width) )
        ratio_h = float(float(h)/float(height) )

        with open(label_path, "r") as f:
            label_lines = f.readlines()
            assert len(label_lines) == 1
            label_items = label_lines[0].split()

        assert img_name == label_items[0]
        xmin = int(label_items[1])
        ymin = int(label_items[2])
        width = int(label_items[3])
        xmax = xmin + width
        height = int(label_items[4])
        ymax = ymin + height




        x1 = float(xmin) * ratio_w
        y1 = float(ymin) * ratio_h
        x2 = float(xmax) * ratio_w
        y2 = float(ymax) * ratio_h




        with open(os.path.join(target_label_path,
                               img_name.split(".")[0] + ".txt"), "w") as f:
            f.write("lpd 0.0 0 0.0 {:.2f} {:.2f} {:.2f} {:.2f} 0.0 0.0 0.0 0.0 0.0 0.0 0.0\n".format(x1, y1, x2, y2))


        # resize images
        image = Image.open(img_path)
        scale_image = image.resize(output_size, Image.ANTIALIAS)
        scale_image.save(os.path.join(target_img_path, img_name))

w = target_width
h = target_height
output_size = (w,h)
img_files = []
for file_name in os.listdir(input_dir):
    if file_name.split(".")[-1] == "jpg":
        img_files.append(file_name)

        image_dir = os.path.join(output_dir, "data")

prepare_data(input_dir, img_files, image_dir, output_size)
 
#print("\nDone. Resized images/labels are saved at {} folder".format(output_dir))

Resizing benchmarks/endtoend/us/wts-lg-000059.jpg and its label
Resizing benchmarks/endtoend/us/wts-lg-000140.jpg and its label
Resizing benchmarks/endtoend/us/wts-lg-000063.jpg and its label
Resizing benchmarks/endtoend/us/wts-lg-000083.jpg and its label
Resizing benchmarks/endtoend/us/0b86cecf-67d1-4fc0-87c9-b36b0ee228bb.jpg and its label
Resizing benchmarks/endtoend/us/wts-lg-000032.jpg and its label
Resizing benchmarks/endtoend/us/us5.jpg and its label
Resizing benchmarks/endtoend/us/wts-lg-000148.jpg and its label
Resizing benchmarks/endtoend/us/wts-lg-000036.jpg and its label
Resizing benchmarks/endtoend/us/wts-lg-000035.jpg and its label
Resizing benchmarks/endtoend/us/wts-lg-000068.jpg and its label
Resizing benchmarks/endtoend/us/us3.jpg and its label
Resizing benchmarks/endtoend/us/us10.jpg and its label
Resizing benchmarks/endtoend/us/wts-lg-000130.jpg and its label
Resizing benchmarks/endtoend/us/wts-lg-000079.jpg and its label
Resizing benchmarks/endtoend/us/wts-lg-000137.

In [10]:
#!mkdir -p $TFRECORDS/lpd && rm -rf $TFRECORDS/lpd/*
!tao detectnet_v2 dataset_convert -d $SPECS_DIR/SPECS_tfrecord.txt -o $TFRECORDS/lpd/


2022-03-05 19:33:41,571 [INFO] root: Registry: ['nvcr.io']
2022-03-05 19:33:41,653 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-tf:v3.21.11-tf1.15.4-py3
2022-03-05 19:33:41,721 [WARNING] tlt.components.docker_handler.docker_handler: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the "/home/pdelafuente/.tao_mounts.json" file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
Using TensorFlow backend.
Using TensorFlow backend.
2022-03-06 00:33:47,022 [INFO] iva.detectnet_v2.dataio.build_converter: Instantiating a kitti converter
2022-03-06 00:33:47,022 [INFO] root: Instantiating a kitti converter
2022-03-06 00:33:47,022 [INFO] root: Generating partitions
2022-03-06 00:33:47,023 [INFO] iva.detectnet_v2.dataio.kitti_converter_lib: Num images in
Train: 178	Val: 44

In [14]:
!tao detectnet_v2 train -e $SPECS_DIR/SPECS_train.txt -r $EXPERIMENT_DIR/exp_unpruned -k nvidia_tlt

2022-03-05 19:41:00,572 [INFO] root: Registry: ['nvcr.io']
2022-03-05 19:41:00,651 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-tf:v3.21.11-tf1.15.4-py3
2022-03-05 19:41:00,718 [WARNING] tlt.components.docker_handler.docker_handler: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the "/home/pdelafuente/.tao_mounts.json" file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
Using TensorFlow backend.
Using TensorFlow backend.

2022-03-06 00:41:06,068 [WARNING] tensorflow: From /opt/tlt/.cache/dazel/_dazel_tlt/75913d2aee35770fa76c4a63d877f3aa/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_wheel.runfiles/ai_infra/iva/detectnet_v2/cost_function/cost_auto_weight_hook.py:43: The name tf.train.SessionRunHook is deprecated. Please use tf.estim

In [20]:
!mkdir -p results/export
!rm -rf $EXPERIMENT_DIR/export/resnet18_detector.etlt
!rm -rf $EXPERIMENT_DIR/export/calibration.bin
!tao detectnet_v2 export \
                  -m $EXPERIMENT_DIR/exp_unpruned/weights/model.tlt \
                  -o $EXPERIMENT_DIR/export/lpd_unpruned_model.etlt \
                  -k $KEY  \
                  --cal_data_file $EXPERIMENT_DIR/export/calibration.tensor \
                  --data_type int8 \
                  --batches 10 \
                  --batch_size 4 \
                  --max_batch_size 4\
                  --engine_file $EXPERIMENT_DIR/export/lpd_detector.trt.int8 \
                  --cal_cache_file $EXPERIMENT_DIR/export/calibration.bin \
                  --verbose

2022-03-05 20:10:44,936 [INFO] root: Registry: ['nvcr.io']
2022-03-05 20:10:45,029 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-tf:v3.21.11-tf1.15.4-py3
2022-03-05 20:10:45,098 [WARNING] tlt.components.docker_handler.docker_handler: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the "/home/pdelafuente/.tao_mounts.json" file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
Using TensorFlow backend.
Using TensorFlow backend.
2022-03-06 01:10:50,624 [INFO] root: Building exporter object.
2022-03-06 01:10:52,239 [INFO] root: Exporting the model.
2022-03-06 01:10:52,239 [INFO] root: Using input nodes: ['input_1']
2022-03-06 01:10:52,240 [INFO] root: Using output nodes: ['output_cov/Sigmoid', 'output_bbox/BiasAdd']
2022-03-06 01:10:52,240 [INFO] iva.common.export